This notebook will let us examine alpha diversity for a category in depth. We're already summarized the alpha diveristy p values for all categories in a [previous notebook](). Here, we can do post-hoc testing and look at the alpha diversity values associated with a single category.

Let's start by importing the modules and functions we'll need to the analysis.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import scipy.stats
import skbio

import americangut.ag_dictionary as agdic
import americangut.diversity_analysis as agdiv
import americangut.notebook_environment as agenv
from americangut.ag_data import AgData

In [ ]:
from matplotlib import rcParams

% matplotlib inline

# Formats the axes using seabron so they will be white, and have ticks
# on the bottom of the axes.
sn.set_style('ticks', {'axes.facecolor': 'none'})

# Sets up plotting parameters so that the default setting is use to Helvetica
# in plots
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Helvetica', 'Arial']
rcParams['text.usetex'] = True

Next, let's select the data set and rarefaction depth we wish to use.

In [ ]:
bodysite = 'fecal'
sequence_trim = '100nt'
rarefaction_depth = '10k'

use_subset = True
use_one_sample = True

Next, we'll load the data, and remove outliers.

In [ ]:
fecal_data = AgData(bodysite=bodysite, 
                    trim=sequence_trim, 
                    depth=rarefaction_depth, 
                    sub_participants=use_subset, 
                    one_sample=use_one_sample)

fecal_data.drop_alpha_outliers()
fecal_data.drop_bmi_outliers()

Next, let's set up a directory where we will save our results.

In [ ]:
save_dir = agenv.check_save_dir(fecal_data.data_set)
fig_dir = os.path.join(save_dir, 'alpha/images/')
tab_dir = os.path.join(save_dir, 'alpha/summary/')

if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
if not os.path.exists(tab_dir):
    os.makedirs(tab_dir)

Finally, we can pick the alpha diversity metric, the category to interogate, and set and order, if one is relevant.

In [ ]:
metric = 'PD_whole_tree'

group_name = 'AGE_CAT'

Now, let's read the files assoicated with the data and load the data dictionary entry for the group.

In [ ]:
group = agdic.ag_dictionary(group_name)

We're going to start by cleaning up the data. So, let's remove any samples that might be outliers (in rounds 1-21, there is a sample with alpha diveristy seven standard deivations above the mean and 4 standard deviations about the next highest sample).

We'll also clean up the mapping column as needed, to make analsyis easier.

Now that we have the data loaded, let's plot it.

In [ ]:
ax = agdiv.pretty_pandas_boxplot(meta=fecal_data.map_,
                                 group=group.name,
                                 metric='%s_%s' % (metric, rarefaction_depth),
                                 order=group.order,
                                 bw=True,
                                 ylim=[5, 55],
                                 xlabel=group.clean_name,
                                 xticklabels=group.order,
                                 ylabel='PD Whole Tree Diversity',
                                 show_p=True,
                                 )
ax.set_position((0.2, 0.2, 0.12*len(group.order), 0.75))

We can also examine significant differences using a post-hoc test to see what drives these differences.

In [ ]:
post_hoc = agdiv.post_hoc_pandas(meta=fecal_data.map_,
                                 group=group.name,
                                 cat='%s_%s'% (metric, rarefaction_depth),
                                 order=group.order,
                                 correct='bonferroni',
                                 )
post_hoc